# Read me
1. To successfully run this code, one has to put all the folders downloaded from kaggle under the folder where this code is being run.
2. Ceate a folder called **path2** where all the pth files will be stored.
3. Run cells one after another to train and test the model

## Model Description
* This is a model composed of 2 convolation layers, followed by one bi-directional lstm with 5 hidden layers and then two fully connected layers.

* Details about the model structure is written in a text cell above the corresponding code cell.

##Acknowledgement: 
Most of the code is from hw3p2_bootcamp. I kinda just filled up the to-do section, and left almost everything else unchanged.

**You can skip the following 4 cells if everything has been properly installed.**

In [ ]:
!kaggle competitions download -c 11785-homework-3-part-2-slack-seq-to-seq
!unzip 11785-homework-3-part-2-slack-seq-to-seq.zip

In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

In [ ]:
!pip install python-Levenshtein

In [ ]:
!pip install jupyter

# **Starting From Here**

In [ ]:
import sys
import time

import Levenshtein

import numpy as np
import pandas as pd


import pdb
import gc
from tqdm.notebook import trange, tqdm

import torch
import torch.nn as nn


from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Check if cuda is available and set device
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

num_workers = 4 if cuda else 0

print("Cuda = ", str(cuda), " with num_workers = ", str(num_workers))

Cuda =  True  with num_workers =  4


In [ ]:
# load training and dev data
train_data = np.load('data/HW3P2_Data/train.npy', allow_pickle=True)
train_labels = np.load('data/HW3P2_Data/train_labels.npy', allow_pickle=True)

dev_data = np.load('data/HW3P2_Data/dev.npy', allow_pickle=True)
dev_labels = np.load('data/HW3P2_Data/dev_labels.npy', allow_pickle=True)

Run the following two cells to see the minimum lengths of labels.

I used these as a guidance for determining the beam_width.


*(As far as I remembered, the minimum length of labels for both training data and validation data is 5, so I chose 3 as my beam_width)*

In [ ]:
min([len(i) for i in train_labels])

In [ ]:
min([len(i) for i in dev_labels])

Note: I pre-permuted the padded x in the dataloader becuase pytorch's CNN expects input to be (N, C${_\text{in}}$, L)

In [ ]:
# Define dataset class
class MyDataSet(Dataset):
  # load the dataset
  def __init__(self, x, y):
    self.X = x
    self.Y = y

  # get number of items/rows in dataset
  def __len__(self):
    return len(self.Y)

  # get row item at some index
  def __getitem__(self, index):
    x = torch.FloatTensor(self.X[index])
    y = torch.LongTensor(self.Y[index])

    return x, y

  def collate_fn(batch):
    (x, y) = zip(*batch)
    x_lens = torch.as_tensor([len(xx) for xx in x])
    y_lens = torch.as_tensor([len(yy) for yy in y])
    x_pad = torch.as_tensor(pad_sequence(x, batch_first=True)).permute(0, 2, 1)#for CNN
    y_pad = torch.as_tensor(pad_sequence(y, batch_first=True))
    return x_pad, y_pad, x_lens, y_lens       

I chose 64 because 128 and 80 both gave me **cuda out of memory error**.

In [ ]:
batch_size = 64 

In [ ]:

# training data
train = MyDataSet(train_data, train_labels)
train_args = {'batch_size': batch_size, 
              'shuffle': True, 'num_workers': num_workers, 
              'collate_fn': MyDataSet.collate_fn, 'pin_memory': True} # TODO: remember to use collate_fn
train_loader = DataLoader(train, **train_args)

# validation data
dev = MyDataSet(dev_data, dev_labels)
dev_args = {'batch_size': batch_size, 
            'shuffle': False, 'num_workers': num_workers, 
            'collate_fn': MyDataSet.collate_fn, 'pin_memory': True}
dev_loader = DataLoader(dev, **dev_args)

First, the input goes through convolution layer with kernel size  = 1, no padding, and stride = 1. 

I decided to start with kernel size = 1 because some forums, like [this one](https://news.ycombinator.com/item?id=17493492), mentioned that 1 by 1 kernel can be used as a pixel wise dense layer, which also acts like a nonlinear function to the neural network.

Then I chose kernel_size  = 3, stride  = 1, padding  = 1 for the second convolution layer, simpy because I didn't want to alter the size of the original data.

After two convolution layers, I added a drop out layer(p  = 0.5) because I noticed that my models (I have been trying with different models) always overfitted the trainning data, I decided to add a drop out layer to prevent overfitting.

I permuted the x again for the input of lstm because it expects input with the size  = (N,L,H)

Then, the input is packed and passed through bidirectional lstm with multiple hidden layers.

Finally, I unpacked the data and passed it through 2 fully-connected layers with one dropout layer following the first activation layer of cnn, and passed the output through LogSoftmax to get probability.

In [ ]:
# TODO: Create model    
class LSTMModel(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, output_size):
    super(LSTMModel, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.output_size = output_size
    #CNN
    self.conv1 = nn.Sequential(
        nn.Conv1d(input_size, hidden_size, kernel_size=1, stride=1, bias=False),
        nn.BatchNorm1d(hidden_size),
        nn.ELU(),)
    self.conv2 = nn.Sequential(
        nn.Conv1d(hidden_size, hidden_size, kernel_size=3, stride=1, padding=1, bias=False),
        nn.BatchNorm1d(hidden_size),
        nn.ELU(),)
    self.dropout = nn.Dropout()
    #lstm
    self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True, dropout=0.5, bidirectional=True)
    self.dense = nn.Sequential(
        nn.Linear(hidden_size*2, hidden_size),
        nn.ELU(),
        nn.Dropout(0.1),
        nn.Linear(hidden_size, output_size),)
    #softmax
    self.logsoftmax = nn.LogSoftmax(dim=2)
  def forward(self, x, lengths):     
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.dropout(x)
    x = x.permute(0, 2, 1)
    x_pack = pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
    x, _ = self.lstm(x_pack)
    x, x_lens = pad_packed_sequence(x, batch_first=True)
    x = self.dense(x)
    x = self.logsoftmax(x)
    #Log_probs: Tensor of size (T, N, C)
    #T=input length, N = batch size C=number of classes
    x = x.permute(1, 0, 2) 
    return x, x_lens

In [ ]:
# create model
input_size = 40
hidden_size = 256
num_layers = 5
output_size = 42

model = LSTMModel(input_size, hidden_size, num_layers, output_size)
model = model.to(device)
#print(model)

The hyperparamerters here are almost the same as the [post ](https://piazza.com/class/knsmz2b3z131mn?cid=1448) suggested.


In [ ]:
# Hyperparams
lr = 9e-3
weight_decay = 5e-6
criterion = nn.CTCLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

# You can add a LR scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3, cooldown=1, verbose=True)

In [ ]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ysie (use `wandb login --relogin` to force relogin)


True

In [ ]:
# Initialize WandB 
config = {
    'batch_size': batch_size,
    'learning_rate': lr,
    'conv1':'kernel_(1,1), stride = 1, padding = 0',
    'conv1':'kernel_(3,3), stride = 1, padding = 1',}
wandb.init(name='#9', 
          project="hw3",  
          config=config)
wandb.watch(model)

[]

In [ ]:
# Train the model
def train_epoch(model, train_loader, criterion, optimizer):
  model.train()

  avg_loss = 0.0
  start = time.time()

  # TODO: Add logic here
  for i, (x, y, x_lens, y_lens) in enumerate(train_loader):
    optimizer.zero_grad()    
    x, y, x_lens, y_lens = x.float().to(device), y.long().to(device), x_lens.to(device), y_lens.to(device)
    out, out_lens = model(x, x_lens)
    loss = criterion(out, y, out_lens, y_lens)
    loss.backward()
    optimizer.step()
    avg_loss += loss
    pbar.update(pbar_num)
    del out, out_lens, y, y_lens, x, x_lens
    gc.collect()
    torch.cuda.empty_cache()    
  end = time.time()
  avg_loss = avg_loss.item()/len(train_loader) # average batch loss
    
  print(f'Training loss: {avg_loss} Time: {end - start}')
  return avg_loss

In [ ]:
import sys
sys.path.append("data/HW3P2_Data")

from phoneme_list import PHONEME_MAP, PHONEME_LIST

In [ ]:
from ctcdecode import CTCBeamDecoder

decoder = CTCBeamDecoder(
    PHONEME_MAP,
    model_path=None,
    alpha=0,
    beta=0,
    cutoff_top_n=42,
    cutoff_prob=1.0,
    beam_width=3,
    num_processes=num_workers,
    blank_id=0,
    log_probs_input=True
)


Note: after getting output from the model, we need to permute the output because decoder.decode expects inputs with size (BATCHSIZE x N_TIMESTEPS x N_LABELS)

In [ ]:
# Train the model
def validate_model(model, val_loader, criterion):

  avg_loss = 0.0
  running_dist = 0.0
  distances = []
  with torch.no_grad():
    # model in validation mode 
    model.eval()

    start = time.time()

    for i, (x, y, x_lens, y_lens) in enumerate(val_loader):

      x, y, x_lens, y_lens = x.float().to(device), y.long().to(device), x_lens.to(device), y_lens.to(device)
      out, output_lens = model(x, x_lens)       
      loss = criterion(out, y, output_lens, y_lens)    
      avg_loss += loss
      
      out = out.permute(1, 0, 2) #BATCHSIZE x N_TIMESTEPS x N_LABELS for decoder.decode input
      #beam_results = n beams of a batch of items.Shape: batchsize x num_beams x num_timesteps
      #out_lens representing the length of each beam in beam_results.Shape: batchsize x n_beams.
      beam_results, beam_scores, timesteps, out_lens = decoder.decode(out, x_lens)
      predict = []
      for i in range(beam_results.size(0)):
        result =  beam_results[i, 0, :out_lens[i][0]] #always pick the top beam result
        phonemes_pred = "".join([PHONEME_MAP[j] for j in result])
        predict.append(phonemes_pred)
      label = []
      for i in range(y.size(0)):
        yy = y[i, :y_lens[i]]
        phonemes = "".join([PHONEME_MAP[j] for j in yy])
        label.append(phonemes)
      
      # Levenshtein
      for pred, target in zip(predict, label):
        distances.append(Levenshtein.distance(pred, target))      
      running_dist += np.mean(distances)
      pbar.update(pbar_num)
      del out, output_lens, y, y_lens, x, x_lens
      torch.cuda.empty_cache()
      gc.collect()
    end = time.time()
    avg_loss = avg_loss.item()/len(val_loader) 
    running_dist /= len(val_loader)
    
    print(f'Validation loss: {avg_loss} Levenshtein distance: {running_dist} Time: {end - start}')    
    return avg_loss, distances, running_dist

In [ ]:
start = 0
best_loss = float('inf')

Don't run the following code until you get 0.34 validation loss. (It happens around 110~120th epoch in my case.) 

We will use that model to run the another 100 epochs with some adjustment on the optimizer and the scheduler first.

In [ ]:
start = 116

In [ ]:
checkpoint = torch.load("path2/"+str(start)+".pth")

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
print("Best validation loss: ", checkpoint['val_loss'])

Best validation loss:  0.34690017700195314


In [ ]:
optimizer.param_groups[0]['lr']

0.000140625

I noticed that when I got to 0.34 for validation loss after first 100ish epochs, the learning rate became really small, and it kept jumping aroud 0.34. 

Then, I tried going with the initial optimizer.(see the code below) It did give me some pretty bad validation loss in the beginning, but somehow once it went down to 0.34 again, the loss kept decreasing, and even gave me better result. (But it takes time)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=2, verbose=True)
best_loss = checkpoint['val_loss']

Please resume running the code here if the model haven't got to the 0.34 validation loss.


Once we get the 0.34 validation loss please stop running (ctrl+c) and run the cells above. (from top to down, we slightly adjusted the code to achieve lower validation loss) Once the scheduler and optimizer have been adjusted, please resume running the code below

In [ ]:
# Define number of epochs
epochs = 200


print('Start...')
pbar_num = 1/(len(train_loader)+len(dev_loader))
with tqdm(total=epochs) as pbar:
    for epoch in range(start+1, start+epochs):
      print('Epoch: ', epoch)

      training_loss = train_epoch(model, train_loader, criterion, optimizer)
      val_loss, distance, running_dist = validate_model(model, dev_loader, criterion)

      # save the best model
      if val_loss < best_loss:
        print('>>>>>>>>>>>>>>>>>>Best loss: {}, epoch: {}'.format(val_loss, epoch))
        # TODO: Save model
        best_loss = val_loss
        torch.save({
        'epoch': epoch,
        'val_loss':val_loss,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()}, 'path2/'+str(epoch)+'.pth')
      wandb.log({'Epoch' :  str(epoch), 
                 'Train Loss' : training_loss,
                 'Val Loss' : val_loss, 
                 'Distances': running_dist})
      scheduler.step(val_loss)
      torch.cuda.empty_cache()
      gc.collect()
      print('='*40)
print('Done...')

Start...


  0%|          | 0/200 [00:00<?, ?it/s]

Epoch:  117


Run the following code once we get a less-than-0.32 validation loss

In [ ]:
best = #put the number with lowest validation loss
checkpoint = torch.load("path2/"+str(best)+".pth")
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
# load test data
test_data = np.load('data/HW3P2_Data/test.npy', allow_pickle=True)

In [ ]:
# Define dataset class
class TestDataSet(Dataset):
  # load the dataset
  def __init__(self, x):
    self.X = x

  # get number of items/rows in dataset
  def __len__(self):
    return len(self.X) 

  # get row item at some index
  def __getitem__(self, index):
    x = torch.FloatTensor(self.X[index])
    return x

  def collate_fn(batch):
    x_len = torch.as_tensor([len(x) for x in batch])
    x_pad = torch.as_tensor(pad_sequence(batch, batch_first=True)).permute(0, 2, 1)
    return x_pad, x_len

In [ ]:
# test data
test = TestDataSet(test_data)
test_args = {'batch_size': batch_size, 
            'shuffle': False, 'num_workers': num_workers, 
            'collate_fn': TestDataSet.collate_fn, 'pin_memory': True}
test_loader = DataLoader(test, **test_args)

In [ ]:
def test_model(model, test_loader):
  predict = []
  with torch.no_grad():
    # model in validation mode 
    model.eval()

    for i, (x, x_lens) in enumerate(test_loader):

      x, x_lens = x.float().to(device), x_lens.to(device)
      out, _ = model(x, x_lens)             
      out = out.permute(1, 0, 2) #BATCHSIZE x N_TIMESTEPS x N_LABELS for decoder.decode input
      #beam_results = n beams of a batch of items.Shape: batchsize x num_beams x num_timesteps
      #out_lens representing the length of each beam in beam_results.Shape: batchsize x n_beams.
      beam_results, beam_scores, timesteps, out_lens = decoder.decode(out, x_lens)      
      for i in range(beam_results.size(0)):
        result =  beam_results[i, 0, :out_lens[i][0]] #always pick the top beam result
        phonemes_pred = "".join([PHONEME_MAP[j] for j in result])
        predict.append(phonemes_pred)
      
      del out, x, x_lens
      torch.cuda.empty_cache()
      gc.collect() 
    return predict

In [ ]:
predictions = test_model(model, test_loader)

In [ ]:
#%%save prediction to csv
import pandas as pd
submit_verification = pd.read_csv("data/HW3P2_Data/sample_submission.csv")

In [ ]:
submit_verification['label']  = predictions

In [ ]:
submit_verification.to_csv('Hw3.csv', index= False)

Double check if we successfully write the result into the sample_submission.csv

In [ ]:
(submit_verification['label']== predictions).all()

Submit the results to Kaggle

In [ ]:
!kaggle competitions submit -c 11785-homework-3-part-2-slack-seq-to-seq -f 'Hw3.csv' -m "Message"

100%|█████████████████████████████████████████| 305k/305k [00:00<00:00, 321kB/s]
Successfully submitted to 11785 Homework 3 Part 2 Slack: Seq to Seq